In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [3]:
path_client_info = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt"
path_exp_web_data1 = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt"
path_exp_web_data2 = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt"
path_test_grouping = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt"

In [4]:
df_client_info = pd.read_csv(path_client_info)

In [5]:
#check for null values
df_client_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [6]:
#dropping nan dominated rows
df_client_info.dropna(thresh=len(df_client_info.columns) - 3,inplace=True)
df_client_info.info()
#14 rows / clients dropped

<class 'pandas.core.frame.DataFrame'>
Index: 70595 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70595 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 5.4+ MB


In [7]:
#load second table, the experiment's web data table 1
df_exp_web_data1 = pd.read_csv(path_exp_web_data1)
df_exp_web_data1
#343141 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [8]:
#load third table, the experiment's web data table 2
df_exp_web_data2 = pd.read_csv(path_exp_web_data2)
df_exp_web_data2
#412264 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [9]:
#merge the two experiment tables via concat:
df_exp_web_data = pd.concat([df_exp_web_data1, df_exp_web_data2], axis=0)
df_exp_web_data
#755405 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [10]:
df_exp_web_data['client_id'].nunique()
#120k clients, why?

120157

In [11]:
#check fourth table of test grouping info per client
df_ab_test_grouping = pd.read_csv(path_test_grouping)
df_ab_test_grouping
#70609 rows -> matches with number of client_ids in df_client_info

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


In [12]:
#check for NaN values:
df_ab_test_grouping.isna().sum()
# 0 NaNs for client_id but 20109 for Variation -> 20k of 70k clients did not
#participate in the test?

client_id        0
Variation    20109
dtype: int64

In [13]:
#Finally cut out the customers that are not participating in the A/B trials
participation_df = df_ab_test_grouping[~df_ab_test_grouping['Variation'].isnull()]
participation_df
#50500 clients left

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
50495,393005,Control
50496,2908510,Control
50497,7230446,Test
50498,5230357,Test


In [14]:
#retrieve client_ids of the 50.500 participating clients and check if apparent in merged web experiment table, if yes
#filter to only keep these for our experiment analysis
participation_filter = participation_df["client_id"].values
df_exp_web_data_filtered = df_exp_web_data[df_exp_web_data['client_id'].isin(participation_filter)]
df_exp_web_data_filtered
#321309 rows left

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
294346,1574008,117364417_77840596075,528720790_71583064618_169151,start,2017-05-06 23:43:27
294391,2908510,814969699_90652851448,562606085_36368381773_92090,start,2017-05-10 22:57:17
294392,2908510,814969699_90652851448,562606085_36368381773_92090,step_2,2017-05-10 22:56:31
294393,2908510,814969699_90652851448,562606085_36368381773_92090,step_1,2017-05-10 22:56:23


In [15]:
#check number of remaining clients
df_exp_web_data_filtered["client_id"].nunique()
#50.500; initially 120157 clients in web_exp data -> 69657 completely new clients within
#the web_exp data but why? are there client_ids in the web_exp that are not in the two other tables?
# the additional 49548 clients will be kicked out by the analysis of the test result anyways, 
#as they dont have a test grouping

50500

In [16]:
#Further EDA & Cleaning: check relationship between client_id, visitor_id and visit_id in web_exp_ table:
df_exp_web_data_filtered

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
294346,1574008,117364417_77840596075,528720790_71583064618_169151,start,2017-05-06 23:43:27
294391,2908510,814969699_90652851448,562606085_36368381773_92090,start,2017-05-10 22:57:17
294392,2908510,814969699_90652851448,562606085_36368381773_92090,step_2,2017-05-10 22:56:31
294393,2908510,814969699_90652851448,562606085_36368381773_92090,step_1,2017-05-10 22:56:23


In [17]:
#merge info about grouping (test vs. control) of clients into web experiment data
df_web_exp = df_exp_web_data_filtered.merge(participation_df, on='client_id', how='left')
df_web_exp['Variation'].value_counts()
#Test       177847
#Control    143462

Variation
Test       177847
Control    143462
Name: count, dtype: int64

In [18]:
df_web_exp.info() #321309 entries
df_web_exp['visitor_id'].nunique() #56011 -> ~500 more than client_id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321309 entries, 0 to 321308
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     321309 non-null  int64 
 1   visitor_id    321309 non-null  object
 2   visit_id      321309 non-null  object
 3   process_step  321309 non-null  object
 4   date_time     321309 non-null  object
 5   Variation     321309 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.7+ MB


56011

In [19]:
#check for visitor_ids with multiple client ids:
#group by 'visitor_id'
grouped = df_web_exp.groupby('visitor_id')

#find groups where 'Variation' has more than one unique value
ambiguous_visitors = grouped.filter(lambda x: x['Variation'].nunique() > 1)

#get unique visitor IDs from confusing groups
ambiguous_visitor_ids = ambiguous_visitors['visitor_id'].unique()

#output the result
print(f"Visitor_id groups with more than one unique 'Variation' value: {len(ambiguous_visitor_ids)}")
#print(f"List of ambiguous visitor group IDs: {ambiguous_visitor_ids}")
#185 ambiguous visitor_ids

Visitor_id groups with more than one unique 'Variation' value: 185


In [20]:
#example check
df_web_exp[df_web_exp['visitor_id'] == '454881699_42713236223']

,client_id,visitor_id,visit_id,process_step,date_time,Variation
3599,7460349,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:14:01,Control
3600,7460349,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:13:30,Control
3601,7460349,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:12:55,Control
3602,7460349,454881699_42713236223,399371316_39189899496_559976,start,2017-04-05 16:12:36,Control
192188,7208557,454881699_42713236223,399371316_39189899496_559976,start,2017-04-05 16:19:36,Test
192189,7208557,454881699_42713236223,399371316_39189899496_559976,step_2,2017-04-05 16:15:33,Test
192190,7208557,454881699_42713236223,399371316_39189899496_559976,step_1,2017-04-05 16:15:03,Test
192191,7208557,454881699_42713236223,399371316_39189899496_559976,start,2017-04-05 16:14:55,Test
304877,7208557,454881699_42713236223,430342294_16750074009_383709,start,2017-05-01 18:24:00,Test
304878,7208557,454881699_42713236223,430342294_16750074009_383709,step_2,2017-05-01 18:20:31,Test


In [21]:
#as they have different variations but the same visitor_id (same device?) and even the same
#visit_id (sessions?) in some cases we have to get rid of these 185 visitor_ids
#from 185 visitor_ids that have more than one variation take the client ids into a drop list
client_ids_to_drop = []
for visitor_id in ambiguous_visitor_ids:
    for i in range(len(df_web_exp[df_web_exp['visitor_id'] == f'{visitor_id}']['client_id'].unique())):
        client_ids_to_drop.append(df_web_exp[df_web_exp['visitor_id'] == f'{visitor_id}']['client_id'].unique()[i])

In [22]:
# Drop rows where client_id is in client_ids_to_drop
df_web_exp = df_web_exp[~df_web_exp['client_id'].isin(client_ids_to_drop)]
#df_web_exp.info() #318872 entries, before 321309 ->3k process steps / timestamps dropped
df_web_exp['visitor_id'].nunique() #55788, before 56011 -> 230 clients dropped

55788

In [23]:
#df_web_exp.to_csv('...',index=False) 
#df_client_info.to_csv('...',index=False)
#df_ab_test_grouping.to_csv('...',index=False)